In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import HTML 
import re

In [2]:
# list of required columns
cols = ['Place of Publication',
 'Author',
 'Date of Publication',
 'Title',
 'Identifier',
 'Flickr URL',
 'Publisher']

# convert all characters in headers to lowercase and change space to underscore
cols_lower = [i.lower().replace(' ','_') for i in cols]

# read only required columns, set index and rename columns
df = pd.read_csv('Datasets/BL-Flickr-Images-Book.csv',
                 usecols=cols,index_col='Identifier').rename(columns = dict(zip(cols,cols_lower)))
df.head()

# for large datasets there are multiple things to think about before loading into a Dataset (memory)
# > do we really want to load all the data in memory?, like above case, we can reduce # of columns to load
# > if the data really needs to be loaded, can we optimize loading the data:
#
#    >> you can specify dtype for the columns in read_csv. This can significantly reduce memory utilization
#       since for e.g. Category type columns will take significantly less memory, since there are only a few 
#       distinct values in column. Like 'author' or 'place_of_publication'. You need to understand the 
#       cardinality of the data columns.
#       checkout: https://towardsdatascience.com/reducing-memory-usage-in-pandas-with-smaller-datatypes-b527635830af
#
#    >> For really large datasets use chunking to break data into smaller sizes or use database to make 
#       querying efficient. Checkout: https://towardsdatascience.com/loading-large-datasets-in-pandas-11bdddd36f7b

,place_of_publication,date_of_publication,publisher,title,author,flickr_url
Identifier,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [3]:
display(HTML('<H3>Count of all columns</H3>'))
df.count()

place_of_publication    8287
date_of_publication     8106
publisher               4092
title                   8287
author                  6509
flickr_url              8287
dtype: int64

In [4]:
display(HTML('<H3>Check cardinality of columns, if you want to use dtypes in read_csv</H3>'))
for col in df:
    cardinality = len(pd.Index(df[col]).value_counts())
    print(f'{df[col].name:<20} : {cardinality:<4}')

place_of_publication : 1441
date_of_publication  : 1148
publisher            : 1989
title                : 8210
author               : 5005
flickr_url           : 8287


In [5]:
display(HTML('<H3>Check if there are any <font color="red">NaN</font> or missing values</H3>'))
for col in df:
    nodata = pd.Index(df[col]).isnull().sum()
    print(f'{df[col].name:<20} : {nodata:<4}')
    
## When you see that there are NaN or missing values, you have to understand that a lot of analytics and
## algorithms dont work well with them. So you have to either convert these values to a standard default
## or remove the rows that have missing data.
## For this you can use:
## > df.dropna() -> to drop the rows containing missing data
## > df.fillna() -> to fill the empty rows with default data

place_of_publication : 0   
date_of_publication  : 181 
publisher            : 4195
title                : 0   
author               : 1778
flickr_url           : 0   


In [6]:
display(HTML('<H3>Cleanup date_of_publication (Year) column</H3>'))
def cleanup_year(year):
    disallowed_characters = '[,-'
    data = str(year)
    if data == 'nan' or data.startswith('['):
        return np.NaN
    else:
        for character in disallowed_characters:
            if character in data:
                character_index = data.find(character)
                data = data[:character_index]
        return data

df['date_of_publication'] = df['date_of_publication'].apply(cleanup_year)
df['date_of_publication'].head()

## Here we are cleaning up column with Year information, there are special characters like [ - , in the value
## we are taking only the component before these special values. For e.g. if the data is 1983-84 we take 1983

Identifier
206    1879 
216     1868
218     1869
472     1851
480     1857
Name: date_of_publication, dtype: object

In [7]:
display(HTML('<H3>Cleanup author (Name) column</H3>'))
def cleanup_name(name):
    author = str(name)
    if author == 'nan':
        return np.NaN
    else:
        author = author.split(',')
        if(len(author)==1):
            name = filter(lambda x:x.isalpha(), author[0])
            return reduce(lambda x,y:x+y,name)
        else:
            lname,fname=author[0],author[1]
            fname = fname[:fname.find('-')] if '-' in fname else fname
            if fname.endswith(('.', '.|')):
                parts = fname.split('.')
                if len(parts) > 1:
                    first_occurence = fname.find('.')
                    final_occurence = fname.find('.', first_occurence + 1)
                    fname = fname[:final_occurence]
            else:
                fname = fname[:fname.find('.')]
        lname = lname.capitalize()
        return f'{fname}, {lname}'

df['author'] = df['author'].apply(cleanup_name)
df['author'].head()

## Here we are cleaning up author name, this is clearly dependant on what is the formatting of the Name. In 
## this case, there is the format looks like "LastName, FirstName.FirstName.OtherStuff" or something similar
## We are splitting it by comma and then cleaning up FirstName to have data before OtherStuff and then joining
## it with the LastName. I have added the comma back again to be able to later break this column into fname and
## lname.

Identifier
206           AA
216     A. A, A.
218     A. A, A.
472     E. S, A.
480     E. S, A.
Name: author, dtype: object

In [8]:
## We can further convert a single author name column to multiple columns using
df[['fname','lname']] = df['author'].str.split(',',expand=True)

In [9]:
display(HTML('<H3>Cleanup title (Name) column</H3>'))
def cleanup_title(title):
    if title == 'nan':
        return 'NaN'
    if title[0] == '[':
        title = title[1: title.find(']')]
    if 'by' in title:
        title = title[:title.find('by')]
    elif 'By' in title:
        title = title[:title.find('By')]
    if '[' in title:
        title = title[:title.find('[')]
    return ' '.join(list(map(str.capitalize, title[:title.find('[')][:-2].split())))

df['title'] = df['title'].apply(cleanup_title)
df['title'].head()

## Here we are cleaning up the title column, againt this depends on the formatting. The idea here is to take
## the value before ('by' or 'By') or between '[' and ']'. We are also Capitalizing the title name.
## Another thing demonstrated here is the chaining of multiple operations, to undersand it you have to look
## from the inside out. The map operation is applied to each word of the title and capitalizes them. Then they
## are joined back together into a sentence.

Identifier
206                                         Walter Forbe
216                                         All For Gree
218                                      Love The Avenge
472    Welsh Sketches, Chiefly Ecclesiastical, To The...
480         The World In Which I Live, And My Place In I
Name: title, dtype: object

In [10]:
display(HTML('<H3>Cleanup place_of_publication (Name) column</H3>'))
pub = df['place_of_publication']
df['place_of_publication'] = np.where(pub.str.contains('London'), 'London',
                                np.where(pub.str.contains('Oxford'), 'Oxford',
                                    np.where(pub.eq('Newcastle upon Tyne'),'Newcastle-upon-Tyne',
                                    df['place_of_publication'])))
df['place_of_publication'].head()

## np.where is like a ternary operator, here we have chained multiple of them. The operation is to only have 
## London,Oxford,Newcastle upon Tyne if the name contains these. Other names of cities are left alone unaltered.

Identifier
206    London
216    London
218    London
472    London
480    London
Name: place_of_publication, dtype: object

In [11]:
display(HTML('<H3>Reading/cleaning a raw file</H3>'))
university_towns = []
failed_regex = []
current_state = ''
with open('Datasets/university_towns.txt', 'r') as file:
    for item in file.readlines():
        state = item.replace('[edit]\n','') if 'edit' in item else ''
        if current_state == '':
            current_state = state
        else:
            if state != '':
                current_state = state                
        if 'edit' not in item:
            regionregex = re.compile(r'(.+?) \((.+?)\)')
            region = regionregex.search(item)
            if((region != None)):
                if(len(region.groups())==2):
                    rlist = [current_state,region.group(1),region.group(2)]
                    university_towns.append(rlist)
            else:
                failed_regex.append([current_state,item])
display(pd.DataFrame(university_towns,columns=['state','city','university']))

## Here we have taken a raw file that contains data that cannot be directly loaded into a dataframe
## You have to parse the data to be able to convert it into a dataframe.
## Format of data looks like this:
## StateName[edit]
## CityName (University/College Name)
## Most of the data has this pattern, however you can see there are somerows that dont match.

,state,city,university
0,Alabama,Auburn,Auburn University
1,Alabama,Florence,University of North Alabama
2,Alabama,Jacksonville,Jacksonville State University
3,Alabama,Livingston,University of West Alabama
4,Alabama,Montevallo,University of Montevallo
...,...,...,...
506,Wisconsin,River Falls,University of Wisconsin–River Falls
507,Wisconsin,Stevens Point,University of Wisconsin–Stevens Point
508,Wisconsin,Waukesha,Carroll University
509,Wisconsin,Whitewater,University of Wisconsin–Whitewater


In [12]:
display(HTML('<H3>Regex Failed for the following lines in raw file</H3>'))
display([line for line in failed_regex])

[['Kentucky',
  'Lexington (University of Kentucky, Transylvania University[5]\n'],
 ['Massachusetts', 'The Colleges of Worcester Consortium:\n'],
 ['Massachusetts', 'The Five College Region of Western Massachusetts:\n'],
 ['Minnesota',
  'Duluth (University of Minnesota Duluth, Lake Superior College, The College of St. Scholastica, University of Wisconsin–Superior, Duluth Business University\n'],
 ['Minnesota', 'Faribault, South Central College\n'],
 ['Minnesota', 'North Mankato, South Central College\n']]

In [32]:
dt = lambda x: pd.to_datetime(x,unit='s')
sp = lambda s: pd.to_numeric(str(s).lstrip('*').rstrip('*'))
dft = pd.read_csv('Datasets/sample.csv',converters={'time':dt,'A1':sp,'A2':sp})
dft


## Another example where we are going to load a csv and convert epoc time to datetime and remove special
## characters from other columns. This uses the "converters" which are similar to map and modify data.
##if you have a float column which has special characters, most of the 
## float operations will faild since they cannot handle the special characters.
## The data in the file looks like this:
## time,A1,A2
##1637681525,1499,1592
##1599423512,*8520*,*9122*

,time,A1,A2
0,2021-11-23 15:32:05,1499,1592
1,2021-03-23 09:21:52,1252,1459
2,2021-11-06 14:08:00,1731,2223
3,2020-03-27 19:27:13,1691,1904
4,2021-12-10 21:27:02,2364,3121
5,2021-12-09 03:50:50,2096,1942
6,2021-11-03 16:10:20,7639,8196
7,2020-12-22 02:26:48,7088,7542
8,2020-10-06 06:50:29,8736,8459
9,2020-12-16 17:37:30,7778,7704


In [33]:
dt = lambda x: pd.to_datetime(x,unit='s')
sp = lambda s: pd.to_numeric(str(s).lstrip('*').rstrip('*'))
dft = pd.read_csv('Datasets/sample.csv',converters={'A1':sp,'A2':sp})
dft['dtime'] = dft['time'].apply(dt)
dft

## Another way is to add an additional column for datetime instead of converting the existing column

,time,A1,A2,dtime
0,1637681525,1499,1592,2021-11-23 15:32:05
1,1616491312,1252,1459,2021-03-23 09:21:52
2,1636207680,1731,2223,2021-11-06 14:08:00
3,1585337233,1691,1904,2020-03-27 19:27:13
4,1639171622,2364,3121,2021-12-10 21:27:02
5,1639021850,2096,1942,2021-12-09 03:50:50
6,1635955820,7639,8196,2021-11-03 16:10:20
7,1608604008,7088,7542,2020-12-22 02:26:48
8,1601967029,8736,8459,2020-10-06 06:50:29
9,1608140250,7778,7704,2020-12-16 17:37:30
